<b><span style="color:#E04728;font-size:26pt">Die Periheldrehung des Merkurs</span><span style="font-size:26pt"> 	&mdash; Analyse</span></b>

Dieses Notebook simuliert die Bewegung des Merkurs, optional unter Berücksichtigung der Relativitätstheorie und verfolgt zusätzlich die Position des Perihels.

# Einrichtung der Simulation

Lade alle nötigen Funktionen und richte Grafikausgabe ein.

In [ ]:
from graphics import *
from analysis import *

Definiere physikalische Parameter. Die folgenden Parameter wurden berechnet mittels https://nssdc.gsfc.nasa.gov/planetary/factsheet

In [ ]:
rM0 = 4.60     # Startradius des Merkurorbits
vM0 = 5.10e-1  # Startgeschwindigkeit des Merkurs
c_a = 9.90e-1  # Basisbeschleunigung des Merkurs
rS  = 2.95e-7  # Schwarzschildradius der Sonne
rL2 = 8.19e-7  # Spezifischer Drehimpuls

Für die physikalischen Berechnungen und grafische Ausgabe brauchen wir Vektoren. Die Startposition und -geschwindigkeit des Merkurs können wie folgt definiert werden.

In [ ]:
vec_rM0 = vector(0, rM0, 0) # Startposition des Merkurs
vec_vM0 = vector(vM0, 0, 0) # Startgeschwindigkeit des Merkurs

Definiere Simulationsparameter. Diese werden nur von der numerischen Simulation gebraucht und haben keine physikalische Bedeutung.

In [ ]:
dt = 2. * vM0 / c_a / 20  # Zeitschritt
max_turns  = 10           # Maximale Anzahl an Umrundungen der Sonne

Die folgenden Parameter bestimmen die Stärke der relativistischen Kraft, siehe Poster.
Hier müssen sehr große Werte gewählt werden, damit in der Simulation etwas zu sehen ist.

In [ ]:
a = 10**6  # Stärke des 1/r**3 tTerms
b = 0      # Stärke des 1/r**4 Terms

# Simuliere den Merkurorbit

In diesem Abschnitt wird der Orbit des Merkurs simuliert und in einer Animation dargestellt.

In [ ]:
def evolve_mercury(vec_rM_old, vec_vM_old, a, b):
    """
    Berechne einen Zeitschritt des Merkurs.
    Argumente:
         - vec_rM_old: alter Positionsvektor des Merkur
         - vec_vM_old: alter Geschwindigkeitsvektor des Merkur
         - a: Stärke des 1/r**3 Terms in der Kraft
         - b: Stärke des 1/r**4 Terms in der Kraft
    Rückgabe:
         - vec_rM_new: neuer Positionsvektor des Merkur
         - vec_vM_new: neuer Geschwindigkeitsvektor des Merkur
    """

    # Berechne den Faktor aus der Allgemeinen Relativitätstheorie
    fact = 1 + a * rS / vec_rM_old.mag + b * rL2 / vec_rM_old.mag**2
    
    # Berechne die absolute Beschleunigung
    aMS = c_a * fact / vec_rM_old.mag**2
    # Multipliziere mit der Richtung um den Beschleunigungsvektor zu erhalten
    vec_aMS = - aMS * (vec_rM_old / vec_rM_old.mag)
    # Aktualisiere Geschwindigkeitsvektor
    vec_vM_new = vec_vM_old + vec_aMS * dt
    # Aktualisiere Positionsvektor
    vec_rM_new = vec_rM_old + vec_vM_new * dt
    
    return vec_rM_new, vec_vM_new

Um die Perihels zu finden und später zu Analysieren werden ein paar zusätzliche Variablen gebraucht.

In [ ]:
vec_r_last = vec_rM0  # Vorherige Position des Merkur
list_perih = list()   # Liste aller gefundenen Perihels

Führe die Simulation aus und finde Perihels.

In [ ]:
# Erzeuge eine neue Grafikausgabe weiter unten
canvas()

# Zeichne Kugeln and den Startpositionen; M = Merkur, S = Sonne
M = sphere(pos=vec_rM0, radius=0.5, color=color.red)
S = sphere(pos=vector(0, 0, 0), radius=1.5, color=color.yellow)
# Setze Startgeschwindigkeiten
M.velocity = vec_vM0
S.velocity = vector(0, 0, 0)

# Mache die Flugbahn des Merkurs sichtbar
M.trajectory = curve(color=color.black, radius=0.02)

# Führe die Simulation for die gegebene Zeit aus und zeichne die Trajektorie
turns = 0  # Anzahl absolvierter Umrundungen der Sonne
while turns < max_turns:
    # Setzte Bildrate
    rate(100)
    # Füge die aktuelle Position zur Trajektorie hinzu
    M.trajectory.append(pos=M.pos)
    # Speicher vorherige und aktuelle Position
    vec_r_before_last = vec_r_last
    vec_r_last = vector(M.pos)
    # Aktualisiere Position und Geschwindigkeit
    M.pos, M.velocity = evolve_mercury(M.pos, M.velocity, a, b)
   
    # Überprüfe ob gerade ein Perihel überschritten wurde
    if vec_r_before_last.mag > vec_r_last.mag < M.pos.mag:
        # Eine weitere Umrundung ist geschafft
        turns = turns + 1
        # Speichere Position des Perihelions
        list_perih.append(vec_r_last)
        
        # Zeichne eine Kugel am Perihel
        sphere(color=color.green, radius=0.2, pos=vec_r_last)

# Analyse

Um Graphen anzuzeigen, muss ein extra Paket geladen werden.

In [ ]:
import matplotlib.pyplot as plt

Berechne die Position des Perihels als Winkel zur Y-Achse damit später die Winkelgeschwindigkeit berechnet werden kann.
Erzeuge außerdem einen Graphen der den Winkel des Perihels in jeder Umrundung der Sonne zeigt.

In [ ]:
# Berechne den Winkel zwischen allen Perihelions und der Y-Achse in Grad.
perih_angles = angles(list_peri)
# Zeichne alle Perihels in einen Graph.
plt.xlabel("Umrundung", size=16)
plt.ylabel("Winkel", size=16)
plt.plot(perih_angles, marker="d")

Bestimme die Winkelgeschwindigkeit aus den gespeicherten Perihels.

In [ ]:
v_perih = (perih_angles[-1] - perih_angles[0]) / len(perih_angles)

M_year = 88.0   # Dauer eines Merkurjahres in Erdentagen
E_year = 365.2  # Dauer eines Erdjahres in Erdentagen
v_perih_years = v_perih / M_year * E_year  # Perihelgeschwindigkeit in Grad pro Erdenjahr (Dreisatz)

print(f"""Der Perihel bewegt sich um {v_perih: .3f}° pro Merkurorbit
                      oder {v_perih_years:.3f}° pro Jahr.""")